In [ ]:
# the finalized exported gradebook from canvas should be stored as grades.csv
# the graded final scores from gradescope should be stored as final.csv
# both files should be in the same directory as this notebook

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
grades = pd.read_csv("grades.csv")
grades = grades.fillna(0)
grades.head()

In [ ]:
# list of assignments, category weight, lowest scores dropped, and maximum score
rq, rq_weight, rq_drops, rq_cap = [], 0.05, 3, 1.0
lect, lect_weight, lect_drops, lect_cap = [], 0.02, 6, 1.0
disc, disc_weight, disc_drops, disc_cap = [], 0.01, 0, 1.0
lab, lab_weight, lab_drops, lab_cap = [], 0.08, 1, 1.0
hw, hw_weight, hw_drops, hw_cap = [], 0.14, 0, 1.0
proj, proj_weight, proj_drops, proj_cap = [], 0.1, 0, 1.0
st, st_weight, st_drops, st_cap = [], 0.15, 1, 1.0

# exams (grouped together when enforcing cap, hence the "dummy" cap)
mt, mt_weight, mt_drops, mt_cap = [], 0.15, 0, 2.0
final, final_weight, final_drops, final_cap = [], 0.3, 0, 2.0

exam_weight, exam_cap = 0.45, 1.0

max_pts = grades.iloc[0]
pid = ["SIS User ID"]

In [ ]:
# place columns in respective categories
for col in grades.columns:
    c = col.lower()
    if c.endswith("score"):
        continue
    elif c.startswith("reading quiz"):
        rq.append(col)
    elif c.startswith("lecture"):
        lect.append(col)
    elif c.startswith("discussion"):
        disc.append(col)
    elif c.startswith("lab"):
        lab.append(col)
    elif c.startswith("hw"):
        hw.append(col)
    elif c.startswith("project"):
        proj.append(col)
    elif c.startswith("skill test"):
        st.append(col)
    elif c.startswith("midterm"):
        mt.append(col)
    elif c.startswith("final"):
        final.append(col)
    elif c.startswith("slip day usage"):
        slip_days = col
    else:
        # print assignments not belonging to any category
        print(col)

In [ ]:
# remove ungraded assignments from categories
lect = [l for l in lect if not l.split()[1] == "0"]
lab = [l for l in lab if not l.split()[0].endswith("00")]

# sanity check to count number of assignments in each category
cats = ["rq", "lect", "disc", "lab", "proj", "st", "final", "mt"]
for cat in cats:
    print(f"{cat}: {len(eval(cat))}", end = " | ")

In [ ]:
# construct final dataframe
out = pd.DataFrame()
out["student"] = grades["Student"][1:-1]
out["pid"] = grades["SIS User ID"][1:-1]
out["sid"] = grades["SIS Login ID"][1:-1]
out.head()

In [ ]:
cats = ["rq", "lect", "disc", "hw", "lab", "proj", "st", "final", "mt"]

for cat in cats:
    # get relevant information for category
    assignments = eval(cat)
    drops = eval(cat + "_drops")
    cap = eval(cat + "_cap")

    # select assignments within category and normalize to proportions
    subset = grades[pid + assignments].iloc[0:-1].copy().set_index(pid)
    for a in assignments:
        subset[a] = subset[a] / subset[a][0]

    # calculate category grade (factoring in drops and caps) 
    total = subset.iloc[0].sum() - drops
    compiled = (subset.sum(axis = 1) - subset.apply(lambda x: x.nsmallest(drops).sum(), axis = 1)) / total
    compiled = pd.DataFrame(np.clip(compiled, 0, cap))
    compiled.columns = [cat + "_score"]

    # add score back to finalized dataframe
    out = out.merge(compiled, left_on = "pid", right_index = True, how = "left")
    
out.head()

In [ ]:
# calculate midterm redemption score from the final
last_mtq = 14 # this is found from the rubric numbering on gradescope

final_df = pd.read_csv("final.csv")
makeup_questions = [c for c in final_df.columns if c.split(":")[0].split(".")[0].isnumeric() and int(c.split(":")[0].split(".")[0]) <= last_mtq]
final_df["mt_redemption"] = final_df[makeup_questions].sum(axis = 1) / max_pts[mt[0]]

out = out.merge(final_df[["SID", "mt_redemption"]], left_on = "pid", right_on = "SID", how = "left").drop(columns = ["SID"])
out.head()

In [ ]:
# DO NOT RUN THIS CELL IF THE FINAL HAS BEEN MOVED TO CANVAS
final_df["final_score"] = final_df["Total Score"] / final_df["Max Points"]
final_df[["SID", "final_score"]]

out = out.drop(columns = ["final_score"]).merge(final_df[["SID", "final_score"]], left_on = "pid", right_on = "SID", how = "left").drop(columns = ["SID"])
out.head()

In [ ]:
# replace midterm score with redemption from final if applicable
out["mt_score"] = out.apply(lambda x: max(x["mt_score"], x["mt_redemption"]), axis = 1)
out.head()

In [ ]:
# combine midterm and final score to calculate exam category
out["exam_score"] = out["mt_score"] * (mt_weight / exam_weight) + out["final_score"] * (final_weight / exam_weight)
out["exam_score"] = np.clip(out["exam_score"], 0, exam_cap)
out.head()

In [ ]:
out = out.fillna(0)

In [ ]:
# calculate grade in the course
def calc_overall(df):
    cats = ["hw", "rq", "lect", "disc", "lab", "proj", "st", "exam"]
    total = [df[cat + "_score"] * eval(cat + "_weight") for cat in cats]
    return sum(total)
    
out["grade"] = out.apply(calc_overall, axis = 1)
out.head()

In [ ]:
# calculate letter grade in the course
def calc_letter_grade(df):
    if df["final_score"] < 0.5:
        return "F"

    grade = df["grade"]
    if grade >= 0.97:
        return "A+"
    elif grade >= 0.93:
        return "A"
    elif grade >= 0.9:
        return "A-"
    elif grade >= 0.87:
        return "B+"
    elif grade >= 0.83:
        return "B"
    elif grade >= 0.8:
        return "B-"
    elif grade >= 0.77:
        return "C+"
    elif grade >= 0.73:
        return "C"
    elif grade >= 0.7:
        return "C-"
    elif grade >= 0.6:
        return "D"
    else:
        return "F"

out["letter_grade"] = out.apply(calc_letter_grade, axis = 1)
out.head()

In [ ]:
# calculate gpa from letter grade
gpa_mapper = {"A+": 4.0, "A": 4.0, "A-": 3.7, "B+": 3.3, "B": 3.0, "B-": 2.7, "C+": 2.3, "C": 2.0, "C-": 1.7, "D": 1.0, "F": 0.0}

out["gpa"] = out.apply(lambda x: gpa_mapper[x["letter_grade"]], axis = 1)
out.head(20)

In [ ]:
out["gpa"].mean()

In [ ]:
out.to_csv("output.csv")